## Gaia Separation

This script finds the accurate distance between two stars.
Method
 - Look up Two Objects by name
 - Find the Gaia DR2 positions and proper motions
 - Project the positions to the relevant epoch
 - Calculate the separation

** Note: Requires Astropy 3. which requires Python 3

In [1]:
from astropy.coordinates import SkyCoord
from astroquery.vizier import Vizier 
import astropy.units as u 
from astropy.time import Time
import astropy
from distutils.version import LooseVersion
import logging
import pdb
if LooseVersion(astropy.__version__) < LooseVersion("3.0"):
    logging.error("Need Astropy >=3.0 for this script. You'll also need Python >3")

In [2]:
c1 = SkyCoord.from_name("KIC 12557548")
c2 = SkyCoord.from_name("KIC 12557592")
obstime = Time('2018-07-16')

Separation without taking into account proper motion

In [3]:
c1.separation(c2).to(u.arcsec)

<Angle 74.35537273 arcsec>

In [4]:
def gaia_query(inCoord, rad_arcsec=0.5, maxmag=35, 
               maxsources=10): 
    """
    Query Gaia DR2 @ VizieR using astroquery.vizier
    ## Adapted from 
    ## https://michaelmommert.wordpress.com/2017/02/13/
        accessing-the-gaia-and-pan-starrs-catalogs-using-python/
    parameters: inCoord: input astropy SkyCoord object
                rad_deg: 
                                          radius in degrees
                maxmag: upper limit G magnitude (optional)
                maxsources: maximum number of sources
    returns: astropy.table object
    """
    ## See http://cdsarc.u-strasbg.fr/viz-bin/Cat?I%2F345
    ## For more on the columns & data table
    columns = ['RA_ICRS','DE_ICRS','pmRA','pmDE','Epoch','Plx']
    vquery = Vizier(row_limit = maxsources,columns=columns)
    rad_deg = rad_arcsec /3600.
    
    queryResult = vquery.query_region(inCoord, 
                                      width=("%fd" % rad_deg), 
                                      catalog="I/345/gaia2")
    if len(queryResult) >= 1:
        table = queryResult[0]
    else:
        logging.error("No Gaia Table Found for {}".format(inCoord))
        return
    
    #pdb.set_trace()
    if table['Epoch'][0] != 2015.5:
        logging.error("Epoch was different from expectations. Stopping")
    else:
        refTime = Time('2015-06-01')
    
    distance = (1.0 * u.pc * u.arcsec / table['Plx']).to(u.pc)
    coordTab = SkyCoord(ra=table['RA_ICRS'],# * u.deg,
                        dec=table['DE_ICRS'],# * u.deg,
                        pm_ra_cosdec=table['pmRA'],# * u.mas/u.yr,
                        pm_dec = table['pmDE'],# * u.mas/u.yr,
                        distance=distance,
                        obstime=refTime)
    
    idx, d2d, d3d = inCoord.match_to_catalog_sky(coordTab)
    
    if d2d > rad_deg * u.deg:
        logging.error("Object not found within {} deg".format(rad_deg))
    else:
        print("Object found within {}".format(d2d.to(u.arcsec)))
        return coordTab[idx]



In [5]:
c1_pm = gaia_query(c1)
c2_pm = gaia_query(c2,rad_arcsec=0.7)

Object found within [0.1564898] arcsec
Object found within [0.25499568] arcsec


In [6]:
newPosC1 = c1_pm.apply_space_motion(new_obstime=obstime)
newPosC2 = c2_pm.apply_space_motion(new_obstime=obstime)

In [7]:
newPosC1.separation(newPosC2).to(u.arcsec)

<Angle 74.10145782 arcsec>